# Model Building

In the last part we created some features for user's data and proposed a hypothesis for our recommendation engine. In this part we will build upon the hypothesis we put forth earlier. We'll pick up one user that satisfy the conditions for each of the hypothesis statement. Let me reiterate the statements:

1. For every expertise level, users with less than average points in the individual domains of FLTA, LTA, and GTA, must be recommended unsolved problems that other users of the domain have solved.

2. For "expert" expertise level, users with less than average points in the individual domain of FGTA, must be recommended unsolved problems that other users of the domain have solved. 

3. For every expertise level, users with greater than average points in the domain of FLTA, must be recommended with problems solved by users of the LTA domain for that expertise level.

4. For every expertise level, users with greater than average points in the domain of LTA, must be recommended with problems solved by users of the GTA domain for that expertise level.

5. For every expertise level, besides the "expert" level, users with greater than average points in the domain of GTA, must be recommended with problems solved by users of the LTA domain for the next expertise level.

6. For every expertise level, besides the "expert" level, users with points in the domain of FGTA must be recommended with problems solved by the users of the GTA domain of next expertise level. 

7. For "expert" level, users with greater than average points in domains of GTA and FGTA must be recommended with unsolved problems that other users of that domain have solved.

First we will build a recommendation engine for all the statements besides 5 & 6. We'll also import a file that consists of test users that satisfy the condition of each of the statements along with all the other processed files.

In [1]:
#Setting the path
setwd("/home/ankit19/Desktop/Jupyter_Notebooks/Recommendation Engine")

#Importing data
UserData = read.csv("data/ProcessedUserData.csv", stringsAsFactors=F)
ProblemData = read.csv("data/ProcessedProblemData.csv", stringsAsFactors=F)
SubmissionsData = read.csv("data/ProcessedSubmissionsData.csv", stringsAsFactors=F)
TestUsers = read.csv("data/TestUsers.csv", stringsAsFactors=F)

TestUsers

user_id,expertise_level,success_rate,points,learner_player,level_of_points,hypothesis_statement
user_2199,beginner,0.67,1500.0,learner,FLTA,1
user_1522,intermediate,0.82,21500.0,learner,LTA,1
user_1911,advanced,0.88,43750.0,player,GTA,1
user_963,expert,0.98,75750.0,learner,FGTA,2
user_1128,expert,0.94,45750.0,learner,FLTA,3
user_2341,intermediate,0.87,28750.0,learner,LTA,4
user_2039,beginner,0.75,22250.0,learner,GTA,5
user_436,advanced,0.97,71662.5,player,FGTA,6
user_842,expert,0.91,68000.0,player,GTA,7
user_1217,expert,0.97,90000.0,player,FGTA,7


Because of the hypothesis we need to handle users differently based on their points. Hence, the recommendation engine can be split functionally into two parts:
    1. An engine that looks for similar users, and similar unsolved problems in the same expertise level.
    2. An engine that looks for problems solved by users of the other expertise level.

The first part applies to users who have fewer than average points in their given domain (FLTA, LTA, GTA) and are hence not close to leveling up. The second part applies to users who have points higher than the average points in their domain and hence are close to leveling up. 

Let's start with building the first engine.

## Engine - I

The operation of this engine can be illustrated by the following flowchart:

<img src="RE1.png">

So let's start building functions for each of the process (depicted in blue boxes). The first process of getting "user_id" is a simple call to a function hence we don't need a separate function for that. We'll start with finding similar users. We'll use user_1522 from TestUsers for the purpose of observing how our functions work.

In [2]:
#Function for finding similar users
findSimilarUsers = function(userID){
    
    #Get user information
    userInfo = subset(UserData, user_id == userID)
    
    #Check if it's a new user i.e. there are no rows
    #If yes, then assign with lowest values for fields
    if(dim(userInfo)[1] == 0){
        userInfo$user_id = userID
        userInfo$expertise_level = "beginner"
        userInfo$success_rate = 0.0
        userInfo$total_points = 0
        userInfo$learner_player = "learner"
        userInfo$level_of_points = "FLTA"
        
        #Indicate that it's a new user
        isColdStart = T
        
        #Assume that user is a learner
        isLearner = T
    }else{
        
        #Not a new user
        isColdStart = F
        if(userInfo$learner_player == "learner"){ isLearner = T }else{ isLearner = F }
    }
    
    #Find users that are either learner or player like the target user,
    #of the same expertise, and level of points
    similarUsers = subset(UserData, 
                          user_id != userInfo$user_id
                          & expertise_level == userInfo$expertise_level
                          & learner_player == userInfo$learner_player
                          & level_of_points == userInfo$level_of_points, 
                          select = c(user_id, total_points))
    
    #Calculate the absolute difference between in points
    #of all other users and target user
    similarUsers$point_diff = abs(similarUsers$total_points - userInfo$total_points)
    
    #Get all users that have least absolute point difference and
    #have points higher or equal to the target user
    similarUsers = subset(similarUsers, 
                          point_diff == min(point_diff)
                          & total_points >= userInfo$total_points,
                          select = c(user_id))
    
    #Remove rownames
    rownames(similarUsers) = NULL
    
    #Combine the values into a list
    similarUsersDetails = list(similarUsers, isColdStart, isLearner)
    names(similarUsersDetails) = c("similarUsers", "isColdStart", "isLearner")
    
    #Return data
    similarUsersDetails
}

In [3]:
#Calling the function
similarUsersDetails = findSimilarUsers("user_1522")
similarUsersDetails$similarUsers
similarUsersDetails$isColdStart
similarUsersDetails$isLearner

user_id
user_3469
user_1707
user_414
user_3431
user_357
user_635
user_2287
user_3346
user_2451
user_1065


[1] FALSE

[1] TRUE

So we see that for user_1522 we get a handful of similar users. The boolean variable serves the purpose of triggering the association rules mining function. We'll get to that in detail later; now let's make the function for finding similar unsolved problems for the user based upon the similar users found.

The logic that will be implemented here is that first we'll look up all the submissions made by the target user and the similar users that we've found. If the boolean value of the variable isLearner is true, then we'll focus on the tags that the unsolved problems have and return those that are similar to the ones the user has solved. On the other hand if the said variable is false, we will simply focus on the difficulty level or level_type of the unsolved problems and return those problems that are similar to the ones that the user has solved.

In [4]:
findUnsolvedProblems = function(userID, similarUsers, isColdStart, isLearner){
    #Check if there are any submissions made by the target user
    if(!isColdStart){
        #Get all submissions made by user
        userSubmissions = subset(SubmissionsData, user_id == userID, select=c(problem_id))
    
        #Get information on all submitted problems
        userSubmissions = subset(ProblemData, problem_id %in% userSubmissions$problem_id)
    
        if(isLearner){
            #Get all unique tags the user's submissions have
            #Tags are split on the comma delimiter and then unlisted
            #Only unique values are stored
            uSub_tags = unique(unlist(strsplit(userSubmissions$tags, ",")))

            #Get all the submissions made by similar users
            #Exclude all those problems that are already solved by target user
            otherSubmissions = subset(SubmissionsData, 
                                      (user_id %in% similarUsers$user_id) & 
                                      !(problem_id %in% userSubmissions$problem_id),
                                      select=c(problem_id))
            
            #Get information on the problems
            otherSubmissions = subset(ProblemData, problem_id %in% otherSubmissions$problem_id, 
                                      select=c(problem_id, tags))
            #Create a dummy column
            otherSubmissions$similar = F

            #For each problem check if their tags match to the tags of user's solved problems
            #If they do, then store TRUE in the "similar" column
            for(i in 1:nrow(otherSubmissions)){
                tags = unique(unlist(strsplit(otherSubmissions[i, "tags"], ",")))
                if(length(intersect(uSub_tags, tags)) > 0){ otherSubmissions[i, "similar"] = T }
            }
            
            similarProblems = subset(otherSubmissions, similar == T, select=c(problem_id, tags))
        }else{
            #Get all the level_types user has attempted
            uSub_levels = levels(factor(userSubmissions$level_type))

            #Get all the submissions made by similar users
            #Exclude all those problems that are already solved by target user
            otherSubmissions = subset(SubmissionsData, 
                                      (user_id %in% similarUsers$user_id) & 
                                      !(problem_id %in% userSubmissions$problem_id),
                                      select=c(problem_id))
            
            #Get information on the problems
            otherSubmissions = subset(ProblemData, problem_id %in% otherSubmissions$problem_id)
            
            #Get all those problems that are at least one of the level_type 
            #that the user has attempted so far
            similarProblems = subset(otherSubmissions, level_type %in% uSub_levels, select=c(problem_id, level_type))
        }
    }else{
        #Get all the submissions made by similar users
        otherSubmissions = subset(SubmissionsData, 
                            (user_id %in% similarUsers$user_id),
                            select=c(problem_id))

        #Get information on the problems
        otherSubmissions = subset(ProblemData, problem_id %in% otherSubmissions$problem_id & level_type == "A", 
                            select=c(problem_id))
        
        #Select just 10 problems
        similarProblems = head(otherSubmissions, 10)
    }

    #Return similar problems
    similarProblems
}

In [5]:
#Calling the function
similarProblems = findUnsolvedProblems("user_1522", 
                                       similarUsersDetails$similarUsers,
                                       similarUsersDetails$isColdStart,
                                       similarUsersDetails$isLearner)
similarProblems

,problem_id,tags
15,prob_3750,math
38,prob_1095,"greedy,shortest paths"
60,prob_226,"brute force,implementation,sortings"
67,prob_3508,implementation
92,prob_3438,implementation
94,prob_2305,"dp,greedy"
113,prob_6362,"binary search,data structures,greedy,sortings"
123,prob_1000,implementation
131,prob_2463,"greedy,implementation,strings"
182,prob_6072,implementation


Above is the list of all similar unsolved problems that the user_1522 may want to consider. But it can be seen that it sure is a long list of problems. We'll try to shorten it. For that, we may need to consider associative rule mining. The reason to consider association as a factor is the assumption that maybe there's a pattern that users across the platform follow. It may be the case the learners on the website follow certain sequence of solving problems as they build upon a topic sequentially. On the other hand, the players may simply follow a sequence that gets them most points and helps them level up faster. It is for this purpose that we may consider finding association rules. 

But although finding association seems desirable, we must not put a lot of value in it. Similar unsolved problems is still a superior set of recommendations as users may not follow any sequence at all! It is because of this we must also consider the possibility that we may not get any association rules at all.

In [6]:
library(dplyr)
library(arules)
findAssocRules = function(userID, similarUsers){
    #Get all submissions made by similar users 
    setOfSubmissions = subset(SubmissionsData, user_id %in% similarUsers$user_id)
    
    #Get user's last submission
    lastSubmission = tail(subset(SubmissionsData, user_id == userID, select = c(problem_id)), 1)
    
    #Compile all the submissions made by each similar user into a single
    #record that resembles a "transaction"
    setOfSubmissions = setOfSubmissions %>% 
                        group_by(user_id) %>% 
                        mutate(problemsByUser = paste0(problem_id, collapse=",")) %>% 
                        select(problemsByUser) %>%
                        distinct()

    #The column of user_id gets added hence we drop it
    setOfSubmissions$user_id = NULL
    
    #To convert dataframe into transaction we need to convert strings to factors
    setOfSubmissions$problemsByUser = as.factor(setOfSubmissions$problemsByUser)
    setOfSubmissions = as(setOfSubmissions, "transactions")
    
    #Generate rules using apriori algorithm with at least 10% support and 70% confidence
    assocRules = apriori(setOfSubmissions, parameter = list(target = "rules", supp = 0.10, conf = 0.7), control = list (verbose=F))
    
    #If some rules are generated
    if(length(assocRules) != 0){
        
        #Generate rules using apriori algorithm with at least 10% support and 70% confidence
        #And also see if the LHS of the rule contains the problem_id of the last submission the user made
        assocRules = apriori(setOfSubmissions, parameter = list(target = "rules", supp = 0.10, conf = 0.7),
                            appearance = list (default="rhs",lhs=lastSubmission$problem_id), control = list (verbose=F))
        
        #Get 10 rules with high confidence
        assocRules = sort (assocRules, by="confidence", decreasing=TRUE)
        possibleProblems = head(data.frame(items = labels(rhs(assocRules))), 10)
        
        #Convert the value of "items" column to character instead of factor
        possibleProblems$items = as.character(possibleProblems$items)
        
        #Remove curly brackets from the first and the last character
        #And select only unique items
        possibleProblems$items = substr(possibleProblems$items, 2, nchar(possibleProblems$items)-1)
        possibleProblems = possibleProblems %>% distinct()
        
        #Change column name to problem_id
        colnames(possibleProblems) = c("problem_id")
        
        #Remove rownames
        rownames(possibleProblems) = NULL
        
        #Indicate if rules are generated
        hasRules = T
    }else{
        #Since no rules are generated we return NULL
        possibleProblems = NULL
        hasRules = F
    }
    
    #Bundle up the outputs into a list
    resultAssocRules = list(hasRules, possibleProblems)
    names(resultAssocRules) = c("hasRules", "possibleProblems")
    
    #Return the results
    resultAssocRules
}

Warning message:
“package ‘dplyr’ was built under R version 3.4.4”
Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: Matrix

Attaching package: ‘arules’

The following object is masked from ‘package:dplyr’:

    recode

The following objects are masked from ‘package:base’:

    abbreviate, write



In [7]:
#Invoking the association rules function
resultAssocRules = findAssocRules("user_1522", similarUsersDetails$similarUsers)
resultAssocRules$hasRules
resultAssocRules$possibleProblems

Warning message:
“package ‘bindrcpp’ was built under R version 3.4.4”Adding missing grouping variables: `user_id`


[1] FALSE

NULL

So as it can be seen we did not get any rules for user_1522. This may be the case for most of the users but we can't be sure and we certainly can't check the same for over 3,000 users! This is why we must handle it with our boolean variable "hasRules". Please note that the warning message is triggered by our dplyr package when we perform the grouping of all submissions by user_id.

Now we will basically merge the three functions we created into one module, which will be a function in itself, that we'll term "RecommendationEngine1". 

In [8]:
RecommendationEngine1 = function(userID){
    #Find information on similar users
    similarUsersDetails = findSimilarUsers(userID)
    
    #Find similar unsolved problems
    similarProblems = findUnsolvedProblems(userID, 
                                       similarUsersDetails$similarUsers,
                                       similarUsersDetails$isColdStart,
                                       similarUsersDetails$isLearner)
    
    #Check if it is a cold start or not
    if(!similarUsersDetails$isColdStart){
        resultAssocRules = findAssocRules(userID, similarUsersDetails$similarUsers)
        
        #Check if any rules were generated
        if(resultAssocRules$hasRules){
            possibleProblems = resultAssocRules$possibleProblems
            
            #If there are less than 5 problems that satisfy the association rules
            #simply return the top 10 similar unsolved problems found
            if(length(intersect(similarProblems$problem_id, possibleProblems$problem_id)) < 5){
                recommendedProblems = head(similarProblems, 10)
                recommendedProblems = subset(ProblemData, problem_id %in% recommendedProblems$problem_id)
                rownames(recommendedProblems) = NULL
            }else{
                #Intersection of problem IDs from both
                recommendedProblems = subset(ProblemData, 
                                             problem_id %in% intersect(similarProblems$problem_id, possibleProblems$problem_id))
                rownames(recommendedProblems) = NULL
            }
        }else{
            #Top 10 similar unsolved problems found 
            recommendedProblems = head(similarProblems, 10)
            recommendedProblems = subset(ProblemData, problem_id %in% recommendedProblems$problem_id)
            rownames(recommendedProblems) = NULL
        }
    }else{
        #We already have 10 problems solved by absolute beginners
        recommendedProblems = subset(ProblemData, problem_id %in% similarProblems$problem_id)
        rownames(recommendedProblems) = NULL
    }
    
    #Return the information of recommended problems
    recommendedProblems
}

We can use this function to recommend problems for any of the users that satisfy the hypothesis statements besides 5 & 6. The function above will thus serve as half of our intended recommendation engine. We'll move on to the next part of the engine in the next IPython Notebook. 